In [1]:
from azureml.core import Workspace, Dataset, Datastore
from azureml.core import Experiment, Environment, Model
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig

In [2]:
from azureml.core.runconfig import RunConfiguration, CondaDependencies, DEFAULT_CPU_IMAGE
from azureml.pipeline.steps import PythonScriptStep, AutoMLStep
from azureml.pipeline.core import Pipeline, PublishedPipeline, PipelineData, TrainingOutput, PipelineRun
from azureml.pipeline.core import StepSequence 
from azureml.widgets import RunDetails

In [3]:
import os

In [4]:
ws = Workspace.from_config()

In [5]:
compute_name = 'compute1548'
compute_target = ComputeTarget(ws, compute_name)

In [6]:
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [7]:
dataset = Dataset.get_by_name(ws, 'Iris Training')

In [8]:
Env = Environment.get(ws,'AutoML Environment')

In [9]:
target_column = 'species'
task = 'classification'
primary_metric = 'accuracy'
featurization = 'auto'
num_classes = 3
iterations = 4

In [10]:
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     num_classes=num_classes,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     max_concurrent_iterations = iterations,
                     n_cross_validations=5,
                     model_explainability=True,
                     enable_stack_ensemble=True,
                     enable_voting_ensemble=True)

In [11]:
metrics_data = PipelineData(name='metrics_data',
                           datastore=datastore,
                           pipeline_output_name='metrics output',
                           training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='model_data',
                           datastore=datastore,
                           pipeline_output_name='best_model_output',
                           training_output=TrainingOutput(type='Model'))

In [12]:
automl_training_step = AutoMLStep(
    name='Multiclass_AutoML_Step',
    automl_config=config,
    outputs=[metrics_data, model_data],
    allow_reuse=False)

In [13]:
os.makedirs('Training_Scripts', exist_ok=True)  

In [14]:
%%writefile Training_Scripts/Iris_Model_Registration.py
from azureml.core import Run, Workspace, Model
from azureml.core import Dataset, Datastore
import argparse

run = Run.get_context()

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", dest="model_name", required=True)
parser.add_argument("--model_path", dest="model_path", required=True)
parser.add_argument("--dataset_name", dest="dataset_name", required=True)
args = parser.parse_args()

def main():  
    ws = run.experiment.workspace

    ds = Dataset.get_by_name(ws, args.dataset_name)
    dataset = [(Dataset.Scenario.TRAINING, ds)]

    model = Model.register(workspace=ws,
                           model_path=args.model_path,
                           model_name=args.model_name,
                           datasets=dataset)
    
if __name__ == '__main__':
    main()

Writing Training_Scripts/Iris_Model_Registration.py


In [15]:
run_config = RunConfiguration()
run_config.environment = Env
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [16]:
model_name = 'Iris-Multi-Classification-AutoML'
dataset_name = 'Iris Training'

In [17]:
model_registration_step = PythonScriptStep(script_name="Iris_Model_Registration.py",
                                       source_directory = 'Training_Scripts',
                                       name="Model-Registration-Step",
                                       allow_reuse=False,
                                       arguments=["--model_name", model_name, "--model_path", model_data,
                                                  "--dataset_name", dataset_name],
                                       inputs=[model_data],
                                       compute_target=compute_target,
                                       runconfig=run_config)

In [18]:
step_sequence = StepSequence(steps=[automl_training_step, model_registration_step])
pipeline = Pipeline(workspace=ws, steps=step_sequence)

In [19]:
# Run your pipeline
pipeline_experiment = Experiment(ws, 'Iris-AutoML-Training-Pipeline-Run')
pipeline_run = pipeline_experiment.submit(pipeline, show_output=True)

Created step Multiclass_AutoML_Step [9410ad96][72229436-5022-4bca-8afa-7cd64a55b273], (This step will run and generate new outputs)
Created step Model-Registration-Step [302f7380][c72dcead-d309-4fb9-8f38-4ff796c8f563], (This step will run and generate new outputs)
Submitted PipelineRun 40f7cdbf-3764-443e-9f99-ab7b8b4c1e7d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/40f7cdbf-3764-443e-9f99-ab7b8b4c1e7d?wsid=/subscriptions/80b2bae0-9f97-4190-a0d1-f57c078442d8/resourcegroups/work/workspaces/mlws-final&tid=7e0f938c-e769-4fbe-8532-f5f38dace694


In [20]:
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 40f7cdbf-3764-443e-9f99-ab7b8b4c1e7d
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/40f7cdbf-3764-443e-9f99-ab7b8b4c1e7d?wsid=/subscriptions/80b2bae0-9f97-4190-a0d1-f57c078442d8/resourcegroups/work/workspaces/mlws-final&tid=7e0f938c-e769-4fbe-8532-f5f38dace694
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 5656a769-6396-4876-9f85-48105bb910d8
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5656a769-6396-4876-9f85-48105bb910d8?wsid=/subscriptions/80b2bae0-9f97-4190-a0d1-f57c078442d8/resourcegroups/work/workspaces/mlws-final&tid=7e0f938c-e769-4fbe-8532-f5f38dace694
StepRun( Multiclass_AutoML_Step ) Status: NotStarted
StepRun( Multiclass_AutoML_Step ) Status: Running

StepRun(Multiclass_AutoML_Step) Execution Summary
StepRun( Multiclass_AutoML_Step ) Status: Finished
{'runId': '5656a769-6396-4876-9f85-48105bb910d8', 'target': 'compute1548', 'status': 'Completed', 'startTimeUtc': '2021-09-05T20:14:06.390437Z', 'endT

'Finished'

In [21]:
published_pipeline = pipeline_run.publish_pipeline(
    name='Iris-AutoML-Training-Pipeline',\
    description='Pipeline that Trains Iris Data with AutoML', version= '1.0')

published_pipeline

Name,Id,Status,Endpoint
Iris-AutoML-Training-Pipeline,8f552da3-5807-4aa9-a6fe-e5ac48caaed6,Active,REST Endpoint


In [23]:
from azureml.pipeline.core import PipelineRun
experiment = Experiment(ws, 'Iris-AutoML-Training-Pipeline-Run')
pipeline_run = PipelineRun(experiment, '40f7cdbf-3764-443e-9f99-ab7b8b4c1e7d')